In [1]:
from modelscope.msdatasets import MsDataset

2023-04-24 16:53:55,727 - modelscope - INFO - PyTorch version 2.0.0 Found.
2023-04-24 16:53:55,731 - modelscope - INFO - Loading ast index from C:\Users\wentao\.cache\modelscope\ast_indexer
2023-04-24 16:53:55,853 - modelscope - INFO - Loading done! Current index file version is 1.5.2, with md5 8c2951608d8ed5827b333a9119fb3c2d and a total number of 860 components indexed


In [2]:
# 载入训练数据，数据格式类似于{'sentence1': 'some content here', 'sentence2': 'other content here', 'label': 0}
train_dataset = MsDataset.load('clue',  subset_name='afqmc', split='train')
# 载入评估数据
eval_dataset = MsDataset.load('clue',  subset_name='afqmc', split='validation')

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset clue downloaded and prepared to C:/Users/wentao/.cache/modelscope/hub/datasets/clue/afqmc/1.0.0/c1dbbd06fe1305b3fd3506775b3f3d575d60f0e4d6833749d39b38adc89ef3cc. Subsequent calls will reuse this data.


Found cached dataset clue (C:/Users/wentao/.cache/modelscope/hub/datasets/clue/afqmc/1.0.0/c1dbbd06fe1305b3fd3506775b3f3d575d60f0e4d6833749d39b38adc89ef3cc)


In [6]:
print(next(iter(train_dataset)))

{'sentence1': '蚂蚁借呗等额还款可以换成先息后本吗', 'sentence2': '借呗有先息到期还本吗', 'label': 0, 'idx': 0}


In [11]:
train_dataset[:3]

{'sentence1': ['蚂蚁借呗等额还款可以换成先息后本吗', '蚂蚁花呗说我违约一次', '帮我看一下本月花呗账单有没有结清'],
 'sentence2': ['借呗有先息到期还本吗', '蚂蚁花呗违约行为是什么', '下月花呗账单'],
 'label': [0, 0, 0],
 'idx': [0, 1, 2]}

In [12]:
# structbert的backbone，该模型没有有效分类器，因此使用前需要finetune（微调）
model_id = 'damo/nlp_structbert_backbone_base_std'

In [13]:
from modelscope.utils.hub import read_config
# 上面的model_id
config = read_config(model_id)
print(config.pretty_text)

2023-04-24 17:08:55,472 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
Downloading: 100%|██████████| 1.13k/1.13k [00:00<00:00, 1.15MB/s]

framework = 'pytorch'
task = 'fill-mask'
model = dict(type='structbert', language='zh')
pipeline = dict(type='fill-mask')
preprocessor = dict(type='fill-mask')
train = dict(
    work_dir='/tmp',
    max_epochs=5,
    dataloader=dict(batch_size_per_gpu=32, workers_per_gpu=1),
    optimizer=dict(type='AdamW', lr=2e-05, options=dict()),
    lr_scheduler=dict(
        type='LinearLR',
        start_factor=1.0,
        end_factor=0.0,
        total_iters=None,
        options=dict(by_epoch=False)),
    hooks=[
        dict(type='CheckpointHook', interval=1),
        dict(type='TextLoggerHook', interval=1),
        dict(type='IterTimerHook'),
        dict(type='EvaluationHook', by_epoch=False, interval=100)
    ])
evaluation = dict(
    dataloader=dict(batch_size_per_gpu=32, workers_per_gpu=1, shuffle=False))



In [18]:
# 这个方法在trainer读取configuration.json后立即执行，先于构造模型、预处理器等组件
def cfg_modify_fn(cfg):
    cfg.preprocessor.type='sen-sim-tokenizer'
    cfg.preprocessor.first_sequence = 'sentence1'
    cfg.preprocessor.second_sequence = 'sentence2'
    cfg.preprocessor.label = 'label'
    cfg.preprocessor.label2id = {'0': 0, '1': 1}
    cfg.model.num_labels = 2
    cfg.task = 'text-classification'
    cfg.pipeline = {'type': 'text-classification'}
    cfg.train.max_epochs = 5
    cfg.train.work_dir = '/tmp'
    cfg.train.dataloader.batch_size_per_gpu = 8
    cfg.evaluation.dataloader.batch_size_per_gpu = 8
    cfg.train.dataloader.workers_per_gpu = 0
    cfg.evaluation.dataloader.workers_per_gpu = 0
    cfg.train.optimizer.lr = 2e-5
    cfg.train.lr_scheduler.total_iters = int(len(train_dataset) / cfg.train.dataloader.batch_size_per_gpu) * cfg.train.max_epochs
    cfg.evaluation.metrics = 'seq-cls-metric'
    # 注意这里需要返回修改后的cfg
    return cfg

In [15]:
from modelscope.trainers import build_trainer

In [24]:
kwargs = dict(
    model = model_id,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    cfg_modify_fn = cfg_modify_fn
)

trainer = build_trainer(default_args=kwargs)

2023-04-24 18:02:45,264 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2023-04-24 18:02:45,888 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2023-04-24 18:02:46,238 - modelscope - INFO - Model revision not specified, use the latest revision: v1.0.0
2023-04-24 18:02:46,414 - modelscope - INFO - initialize model from C:\Users\wentao\.cache\modelscope\hub\damo\nlp_structbert_backbone_base_std
Some weights of the model checkpoint at C:\Users\wentao\.cache\modelscope\hub\damo\nlp_structbert_backbone_base_std were not used when initializing SbertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight']
- This IS expected if you are

In [ ]:
trainer.train()

2023-04-24 18:07:09,373 - modelscope - WARNING - ('OPTIMIZER', 'default', 'AdamW') not found in ast index file
2023-04-24 18:07:09,375 - modelscope - WARNING - ('LR_SCHEDULER', 'default', 'LinearLR') not found in ast index file
2023-04-24 18:07:09,376 - modelscope - INFO - Stage: before_run:
    (ABOVE_NORMAL) OptimizerHook                      
    (LOW         ) LrSchedulerHook                    
    (LOW         ) CheckpointHook                     
    (VERY_LOW    ) TextLoggerHook                     
 -------------------- 
Stage: before_train_epoch:
    (LOW         ) LrSchedulerHook                    
 -------------------- 
Stage: after_train_iter:
    (ABOVE_NORMAL) OptimizerHook                      
    (NORMAL      ) EvaluationHook                     
    (LOW         ) LrSchedulerHook                    
    (LOW         ) CheckpointHook                     
    (VERY_LOW    ) TextLoggerHook                     
 -------------------- 
Stage: after_train_epoch:
    (NORMA

In [1]:
import torch

In [2]:
torch.cuda.is_available()

True